# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [210]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
# https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=NhTpQ98aX0E%3D

In [211]:
driver = webdriver.Chrome()
driver.get('https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=NhTpQ98aX0E%3D')

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [212]:
agree = driver.find_element_by_id('checkbox')
agree.click()

enter_site = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_site.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [213]:
search_records = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_records.click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [214]:
jurisdiction = Select(driver.find_element_by_id('slcJurisdiction'))
jurisdiction.select_by_visible_text('Statewide')

In [215]:
trade = driver.find_element_by_id('txtTradeName')
trade.send_keys('Vap%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [216]:
table = driver.find_element_by_class_name('searchtable')
table.submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [217]:
business_names = driver.find_elements_by_class_name("searchlistitem")
for name in business_names:
    print(name.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [199]:
business_names = driver.find_elements_by_class_name("searchfieldtitle")
business_names

for name in business_names:
    print(name.find_elements_by_tag_name('td')[1].text)
    try:
        link = name.find_element_by_tag_name('a')
        print(link.get_attribute('href'))
    except:
        print("License pending.")

VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=5Bluh%2Bj4cqc%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wTRqdIacr20%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=zKUoVrzuIo0%3D
VAPE FROG
License pending.
VAPE FROG
License pending.


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [157]:
business_names = driver.find_elements_by_class_name("searchfieldtitle")
for name in business_names:
    print(name.find_elements_by_tag_name('td')[1].text)
    try:
        link = name.find_element_by_tag_name('a')
        link.click()  
    
        # this has both mailing and license info
        info = driver.find_element_by_class_name('tablecelltext').text
        
        # mailing address
        mailing = info.split('\n')[1:3]
        print(mailing)
        
        # location address
        location = driver.find_elements_by_class_name('tablecelltext')[1].text
        print(location)
        
        # license information
        print(info.split('\n')[3:9])
        
        # total amount paid
        amount_paid = driver.find_elements_by_class_name('tablecelltext')[7].text
        print(amount_paid)
        
        # issued by
        issued_by = driver.find_elements_by_class_name('tablecelltext')[8].text
        print(issued_by)
        
        # licenses
        # This prints all of the body text -- needs to be revised
        #licenses = driver.find_elements_by_tag_name('tbody')[2].text
        #print(licenses)
        
        
        print("yay")
        
        # return to search
        back_to_search = driver.find_element_by_partial_link_text('window.history.back();')
        back_to_search.click()
    except:
        print("License pending.")

VAPE IT STORE II
['1104 PLANTERS PLACE', 'SALISBURY, MD 21804 License Status: Issued']
VAPE IT STORE II
1015 S SALISBURY BLVD
SALISBURY, MD 21801
['License No.: 22375606', 'Control No.: 22884439', 'Date of Issue: 4/27/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 22 Salisbury']
Amount Paid: $ 41.00
WICOMICO COUNTY, CLERK OF CIRCUIT COURT
P.O. BOX 198
SALISBURY, MARYLAND 21803-0198
yay
License pending.


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [143]:
rows = []
for name in business_names:
    vape_shops = {}
    try:
        vape_shops['name'] = name.text
        #print(name.text)
        link = name.find_element_by_tag_name('a')
        vape_shops['url'] = link.get_attribute('href')
        #print(link.get_attribute('href'))
        
    except:
        pass
    
    rows.append(vape_shops)
print(rows)

[{'name': '1. VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=C2i1u84sNKM%3D'}, {'name': '2. VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=hXrft%2B65aKs%3D'}, {'name': '3. VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=TtuzVObjAb4%3D'}, {'name': '4. VAPE FROG Pending *'}, {'name': '5. VAPE FROG Pending *'}]


In [144]:
df = pd.DataFrame(rows)
df

,name,url
0,1. VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,2. VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,3. VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,4. VAPE FROG Pending *,NaN
4,5. VAPE FROG Pending *,NaN


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [145]:
rows = []
for name in business_names:
    vape_shops = {}
    
    vape_shops['name'] = name.text
    print(name.text)
    try:
        link = name.find_element_by_tag_name('a')
        vape_shops['url'] = link.get_attribute('href')
        
    except:
        pass
    
    rows.append(vape_shops)
print(rows)

# scroll to the "Next" button and click it
next_button = driver.find_element_by_partial_link_text('Next')
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()

print("Scraping a new page.")

1. VAPE IT STORE II
2. VAPE IT STORE I
3. VAPEPAD THE
4. VAPE FROG Pending *
5. VAPE FROG Pending *
[{'name': '1. VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=C2i1u84sNKM%3D'}, {'name': '2. VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=hXrft%2B65aKs%3D'}, {'name': '3. VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=TtuzVObjAb4%3D'}, {'name': '4. VAPE FROG Pending *'}, {'name': '5. VAPE FROG Pending *'}]
Scraping a new page.


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [190]:
rows = []

while True:

    for name in business_names:
        vape_shops = {}
        try:
            vape_shops['name'] = name.text
            link = name.find_element_by_tag_name('a')
            vape_shops['url'] = link.get_attribute('href')

        except:
            pass

        rows.append(vape_shops)

    next_button = driver.find_element_by_partial_link_text('Next')
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()

    print('Scraping a new page.')  
print(rows)

Scraping a new page.
Scraping a new page.
Scraping a new page.
Scraping a new page.
Scraping a new page.
Scraping a new page.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.4 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [219]:
rows = []

try:
    while True:
        business_names = driver.find_elements_by_class_name("searchfieldtitle")
        for name in business_names:
            vape_shops = {}
            try:
                vape_shops['name'] = name.find_elements_by_tag_name('td')[1].text
                link = name.find_element_by_tag_name('a')
                vape_shops['url'] = link.get_attribute('href')

            except:
                pass

            rows.append(vape_shops)

        next_button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()

        print('Scraping a new page.')  

except:
    pass

print(rows)

Scraping a new page.
Scraping a new page.
Scraping a new page.
Scraping a new page.
Scraping a new page.
Scraping a new page.
[{'name': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ig26vbd8TYw%3D'}, {'name': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=84gQyyJA0pc%3D'}, {'name': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ExxQt2Az02g%3D'}, {'name': 'VAPE FROG'}, {'name': 'VAPE FROG'}, {'name': 'VAPE LOFT THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=oOIqfPHef0Q%3D'}, {'name': 'VAPE N CIGAR', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kA2cxN0367s%3D'}, {'name': 'VAPE DOJO', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ArE2JjRrNHI%3D'}, {'name': 'VAPE HAVEN'}, {'name': 'VAPORS LOUNGE'}, {'name': 'VAPORFI SOUTHSIDE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=r

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [220]:
df = pd.DataFrame(rows)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


In [221]:
df.to_csv('vape-shops-basic.csv', index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [222]:
pd.set_option('display.max_colwidth', -1)
df.head(1)

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ig26vbd8TYw%3D


### Use Selenium to visit that page

In [233]:
driver.get(df['url'][0])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [238]:
# all of the 'print' fucntions were for testing the rows

rows = []

business_names = driver.find_elements_by_class_name("searchfieldtitle")
vape_shops = {}

try:

    # this has both mailing and license info
    info = driver.find_element_by_class_name('tablecelltext').text

    # mailing address
    #print(info.split('\n')[1:3])
    vape_shops['mailing_address'] = info.split('\n')[1:3]

    # location address
    #print(driver.find_elements_by_class_name('tablecelltext')[1].text)
    vape_shops['location_address'] = driver.find_elements_by_class_name('tablecelltext')[1].text

    # license information
    #print(info.split('\n')[3:9])
    vape_shops['license_information'] = info.split('\n')[3:9]

    # total amount paid
    #print(driver.find_elements_by_class_name('tablecelltext')[7].text)
    vape_shops['total_amount_paid'] = driver.find_elements_by_class_name('tablecelltext')[7].text

    # issued by
    #print(driver.find_elements_by_class_name('tablecelltext')[8].text)
    vape_shops['issued_by'] = driver.find_elements_by_class_name('tablecelltext')[8].text

    print("yay!")


except:
    print("No good!")

print(vape_shops)

yay!
{'mailing_address': ['1104 PLANTERS PLACE', 'SALISBURY, MD 21804 License Status: Issued'], 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801', 'license_information': ['License No.: 22375606', 'Control No.: 22884439', 'Date of Issue: 4/27/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 22 Salisbury'], 'total_amount_paid': 'Amount Paid: $ 41.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}


### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [239]:
driver.get(df['url'][0])

rows = []

business_names = driver.find_elements_by_class_name("searchfieldtitle")
vape_shops = {}

try:
    # this has both mailing and license info
    info = driver.find_element_by_class_name('tablecelltext').text

    # mailing address
    vape_shops['mailing_address'] = info.split('\n')[1:3]

    # location address
    vape_shops['location_address'] = driver.find_elements_by_class_name('tablecelltext')[1].text

    # license information
    vape_shops['license_information'] = info.split('\n')[3:9]

    # total amount paid
    vape_shops['total_amount_paid'] = driver.find_elements_by_class_name('tablecelltext')[7].text

    # issued by
    vape_shops['issued_by'] = driver.find_elements_by_class_name('tablecelltext')[8].text

    print("yay!")


except:
    print("No good!")

print(vape_shops)

yay!
{'mailing_address': ['1104 PLANTERS PLACE', 'SALISBURY, MD 21804 License Status: Issued'], 'location_address': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801', 'license_information': ['License No.: 22375606', 'Control No.: 22884439', 'Date of Issue: 4/27/2018', 'Months Paid: 12', 'Exp. Date: 4/30/2019', 'Subdivision: 22 Salisbury'], 'total_amount_paid': 'Amount Paid: $ 41.00', 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}


### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [278]:
def get_that_vape_info(row):

# defining the variables here so the function doesn't break on shops with no URLs
    mailing_address = ''
    location_address = ''
    license_information = ''
    total_amount_paid = ''
    issued_by = ''
    
    try:
        driver.get(row['url'])

        # this has both mailing and license info
        info = driver.find_element_by_class_name('tablecelltext').text

        # mailing address
        mailing_address = info.split('\n')[1:3]

        # location address
        location_address = driver.find_elements_by_class_name('tablecelltext')[1].text

        # license information
        license_information = info.split('\n')[3:9]

        # total amount paid
        total_amount_paid = driver.find_elements_by_class_name('tablecelltext')[7].text
        
        # issued by
        issued_by = driver.find_elements_by_class_name('tablecelltext')[8].text

    except:
        pass

    return pd.Series({
        'mailing_address': mailing_address,
        'location_address': location_address,
        'license_information': license_information,
        'total_amount_paid': total_amount_paid,
        'issued_by': issued_by
    })
    

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [279]:
new_df = df.apply(get_that_vape_info, axis=1).join(df)
new_df.head(3)

,mailing_address,location_address,license_information,total_amount_paid,issued_by,name,url
0,"[1104 PLANTERS PLACE, SALISBURY, MD 21804 License Status: Issued]","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801","[License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]",Amount Paid: $ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ig26vbd8TYw%3D
1,"[1104 PLANTERS PLACE, SALISBURY, MD 21804 License Status: Issued]","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801","[License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=84gQyyJA0pc%3D
2,"[2504 ORCHARD KNOLL WAY, ODENTON, MD 21113 License Status: Issued]","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054","[License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ExxQt2Az02g%3D


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [280]:
new_df

,mailing_address,location_address,license_information,total_amount_paid,issued_by,name,url
0,"[1104 PLANTERS PLACE, SALISBURY, MD 21804 License Status: Issued]","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801","[License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]",Amount Paid: $ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ig26vbd8TYw%3D
1,"[1104 PLANTERS PLACE, SALISBURY, MD 21804 License Status: Issued]","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801","[License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=84gQyyJA0pc%3D
2,"[2504 ORCHARD KNOLL WAY, ODENTON, MD 21113 License Status: Issued]","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054","[License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ExxQt2Az02g%3D
3,,,,,,VAPE FROG,NaN
4,,,,,,VAPE FROG,NaN
5,"[License Status: Issued, License No.: 02310799]","VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWATER, MD 21037","[Control No.: 02686069, Date of Issue: 4/03/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=oOIqfPHef0Q%3D
6,"[License Status: Issued, License No.: 13343011]","VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045","[Control No.: 13856368, Date of Issue: 4/30/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 13 Howard County]","WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045",,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=kA2cxN0367s%3D
7,"[License Status: Issued, License No.: 06327188]","VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15\nWESTMINSTER, MD 21157","[Control No.: 06946760, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 06 Westminster]","DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155",,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ArE2JjRrNHI%3D
8,,,,,,VAPE HAVEN,NaN
9,,,,,,VAPORS LOUNGE,NaN


In [281]:
new_df.to_csv('vape-total.csv', index = False)